In [1]:
using DataFrames, XLSX, DataFramesMeta, DataStructures;
using LinearAlgebra, HDF5;
using JuMP,Ipopt;

In [2]:
TestData = DataFrame(From = [ 1, 1, 2, 3, 3, 4], To = [ 2, 3, 4, 4, 5, 5], 
    R = [ 0.0, 0.023, 0.006, 0.020, 0.0, 0.0], X = [ 0.3, 0.145, 0.032, 0.260, 0.320, 0.500], 
    Bsh = [ 0.0, 0.040, 0.010, 0.0, 0.0, 0.0], T = [ 1.0, 1.0, 1.0, 1.0, 0.98,1], 
    ϕ = [ 0.0, 0.0, 0.0, -3.0, 0.0, 0.0]);
Zsh = [ 0.0+0.0*im 0.0+0.3*im 0.05+0.0*im 0.0+0.0*im 0.0+0.0*im]; 
BusNum = 5

5

In [3]:
R = TestData.R; X = TestData.X; Bsh = TestData.Bsh; T = TestData.T; ϕ = TestData.ϕ;
Y = (R + X*im).^-1; G = real(Y); B = imag(Y);
insertcols!(TestData, 3, Y = Y);

In [4]:
Yd = Dict{Tuple{Int64, Int64}, Complex{Float64}}();   # Admittance Matrix
Bshd = Dict{Tuple{Int64, Int64}, Float64}(); # Shunt Susceptance Dictionary
Td = Dict{Tuple{Int64, Int64}, Float64}();   # Nominal Transformer
ϕd = Dict{Tuple{Int64, Int64}, Float64}();   # Transformer Phase Shift
Y =  Dict{Tuple{Int64, Int64}, Complex{Float64}}(); # Diagnol ELements of Admittance Matrix

### Aquire Line Parameters from Data Frame

In [5]:
for i in 1:size(TestData)[1]
    Yd[(TestData.From[i], TestData.To[i])] = TestData.Y[i]
    Bshd[(TestData.From[i], TestData.To[i])] = TestData.Bsh[i]
    Td[(TestData.From[i], TestData.To[i])] = TestData.T[i]
    ϕd[(TestData.From[i], TestData.To[i])] = TestData.ϕ[i]
end

### Define Diagnol Elements

In [6]:
for i in unique(append!(TestData.From,TestData.To))
Y_N = filter(((k1,k2),v) -> k1 == i || k2 == i,Yd);
T_N = filter(((k1,k2),v) -> k1 == i || k2 == i,Td);
ϕ_N = filter(((k1,k2),v) -> k1 == i || k2 == i,ϕd);

Y[i,i] = sum(
        append!([1/( round(T_N[i,j]*exp( deg2rad(ϕ_N[i,j])*im),digits = 3) * round(conj(T_N[i,j]*exp( deg2rad(ϕ_N[i,j])*im)), digits = 3) ) * ( (Bshd[i,j]*im)/2 + Y_N[i,j]) for (i,j) in keys(Y_N)],Zsh[i]))


end

┌ Warning: In `filter(f, dict)`, `f` is now passed a single pair instead of two arguments.
│   caller = top-level scope at In[6]:2
└ @ Core In[6]:2
┌ Warning: In `filter(f, dict)`, `f` is now passed a single pair instead of two arguments.
│   caller = top-level scope at In[6]:3
└ @ Core In[6]:3
┌ Warning: In `filter(f, dict)`, `f` is now passed a single pair instead of two arguments.
│   caller = top-level scope at In[6]:4
└ @ Core In[6]:4


### Define Off-Diagnol Elements

In [7]:
for (i,j) in keys(Yd)
Y[i,j] =- 1/conj(Td[i,j]*exp( deg2rad(ϕd[i,j]*im)) ) * (Yd[i,j])
end


### Create Matrix for Checking Results

In [8]:
Y_Matrix = zeros(Complex{Float64}, 5, 5)
for (i,j) in keys(Y) Y_Matrix[i,j] = Y[i,j] end
for (i,j) in keys(Y) Y_Matrix[j,i] = Y[i,j] end

In [9]:
show(IOContext(stdout, :limit=>false), MIME"text/plain"(), Y_Matrix)

5×5 Array{Complex{Float64},2}:
  1.06709-10.0406im      -0.0+3.33333im    -1.06709+6.72729im         0.0+0.0im       0.0+0.0im    
     -0.0+3.33333im   5.66038-33.217im          0.0+0.0im        -5.66038+30.1887im   0.0+0.0im    
 -1.06709+6.72729im       0.0+0.0im           1.411-13.782im   -0.0936065+3.83368im  -0.0+3.18878im
      0.0+0.0im      -5.66038+30.1887im  -0.0936065+3.83368im     5.95429-36.0045im  -0.0+2.0im    
      0.0+0.0im           0.0+0.0im            -0.0+3.18878im        -0.0+2.0im       0.0-5.25385im

In [ ]:
Y_Matrix[1,1]

In [ ]:
ϕd

In [10]:
Bus = DataFrame(P = [ 0.0, 0.0, 0.0, 0.9, 0.239], Q = [ 0.0, 0.0, 0.0, 0.4, 0.129], 
    Vmin = [ 1.0, 0.95, 0.95, 0.95, 0.95], Vmax = [ 1.0, 1.05, 1.05, 1.05, 1.05]);
Gen = DataFrame(Bus = [1,3,4],
                a = [0.0, 0.40, 0.50], 
                b = [0.35, 0.20, 0.30],
                c = [0, 0, 0],
                Pmin = [ -Inf, 0.10, 0.05], 
                Pmax = [ Inf, 0.4, 0.4], 
                Qmin = [ -Inf, -0.20, -0.20], 
                Qmax = [ Inf, 0.3, 0.2]);

In [11]:
GenD =  DefaultDict(Dict)

DefaultDict{Any,Any,UnionAll} with 0 entries

In [12]:
for i in 1:length(Gen.Bus)
    
    for j in names(Gen)[2:end]

    GenD[Gen.Bus[i]][string(j)] = Gen[i,j];

    end
end

In [ ]:
GenD[1]["a"]

In [ ]:
for i in keys(GenD)
    println(keys(GenD[i]))
end

In [ ]:
for i in keys(GenD)
    for j in keys(GenD[i])
        println("(",i," , ", j,") ", GenD[i][j])
    end
end

In [21]:
model = nothing
model = Model(with_optimizer(Ipopt.Optimizer))

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Ipopt

In [22]:
@variable(model, Pg[Gen.Bus]);
@variable(model, V[1:BusNum]);
@variable(model, P[1:BusNum]);
@variable(model, Q[1:BusNum]);
@variable(model, δ[1:BusNum]);
@variable(model, T[1:BusNum,1:BusNum]);
@variable(model, ϕ[1:BusNum,1:BusNum]);
@variable(model, L[Gen.Bus]);

In [23]:
@objective(model,Min, sum( GenD[i]["a"]*(Pg[i]^2) + GenD[i]["b"]*Pg[i] + GenD[i]["c"]  for i in Gen.Bus))

0 Pg[1]² + 0.4 Pg[3]² + 0.5 Pg[4]² + 0.35 Pg[1] + 0.2 Pg[3] + 0.3 Pg[4]

In [ ]:
@constraint( L[i] - Pg[i] == V[i]*sum(V[k]*( *cos(δ[i] - δ[k]))) ,name = "Active Power Flow")

In [19]:
real(Y[(1,1)])

1.067087315579475